In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from bin import *

from IPython.display import clear_output
%load_ext autoreload
%autoreload 2

In [2]:
mpl.use("pgf")

mpl.rcParams.update({
    "pgf.texsystem": "lualatex",
    'font.family': 'serif',
    'font.size' : 8,
    'text.usetex': True,
    'pgf.rcfonts': False,
    'pgf.preamble': tex_preamble,
})

In [3]:
n = 100
Λ = model_problem_spectrum(n,.6,1e3)
λmin = np.min(Λ)
λmax = np.max(Λ)

A = np.diag(Λ)

b = np.ones(n)
b /= np.linalg.norm(b)

f = lambda x: np.exp(-x)

fAb = np.diag(f(Λ))@b

In [4]:
def lanczos_FA_bad(f,Q,b,α,β,k,normb=1):
    
    if k==0:
        return np.zeros_like(Q[:,0])
        
    Θ,S = sp.linalg.eigh_tridiagonal(α[:k],β[:k-1],tol=1e-30)
    
    return Q[:,:k]@(S@(f(Θ)*(S.T)))@Q[:,:k].T@b

In [5]:
k_max = 44
Q,(α,β) = lanczos_reorth(A,b,k_max,reorth=0)

err_FA = np.full(k_max,np.nan)
for k in range(k_max):
    err_FA[k] = np.linalg.norm(fAb - lanczos_FA(f,Q,α,β,k))


err_FA_bad = np.full(k_max,np.nan)
for k in range(k_max):
    err_FA_bad[k] = np.linalg.norm(fAb - lanczos_FA_bad(f,Q,b,α,β,k))

In [6]:
fig = plt.figure(figsize=(figure_width*cm,figure_height*cm))

ax = fig.add_axes([left, bottom, width, height])

ax.plot(err_FA_bad,**line_styles['l2'],label='no reorth')
ax.plot(err_FA,**line_styles['l1'],label='no reorth')

ax.grid(True,linestyle=':',linewidth=.5)

ax.set_xlabel('number of matvecs: $k$')
ax.set_ylabel('error norm')
ax.set_yscale('log')

plt.savefig('imgs/QfTQv_QfTe1.pdf')
plt.close()